In [1]:
import hashlib
import json
import datetime
from flask import Flask, jsonify
import pandas as pd

/var/folders/j6/fhk9cwt11v5bj1s7_3dhhb8h0000gn/T/ipykernel_58172/415521437.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import subprocess
import sys

def install_sk():
    subprocess.check_call([sys.executable, "-m", "pip","install", "--upgrade", "psycopg2"])
try:
    import psycopg2
except:
    install_sk()
    import psycopg2

In [13]:
database

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,Иванова Мария Ивановна,1111111111,111111111,ГБОУ Школа 1111,1111111111,111111111,111111111,5000,2024-01-01,40603810450000000005,30101810145250000411,40603810450000000005,30101810145250000411


In [5]:
def connect_db():
    conn = psycopg2.connect(dbname='postgres', user='postgres', password='qwerty123', host='localhost', port='5432')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM "data_for_blockchain"')
    df = cursor.fetchall()
    df = pd.DataFrame(df)
    return df

database = connect_db()



In [15]:
database[8].

5000

In [17]:
class Blockchain:
# Эта функция ниже создана для создания самого первого блока и установки его хэша равным "0"
   def __init__(self):
       self.chain = []
       self.create_block(proof=1, previous_hash='0')
# Эта функция ниже создана для добавления дополнительных блоков в цепочку
   def create_block(self, proof, previous_hash):
       block = {'index': len(self.chain) + 1,
                'timestamp': str(datetime.datetime.now()),
                'fio_payer': str(database[1].iloc[0]),
                'inn_payer': str(database[2].iloc[0]),
                'bik_payer': str(database[3].iloc[0]),
                'name_recep': str(database[4].iloc[0]),
                'inn_recep': str(database[5].iloc[0]),
                'kpp_recep': str(database[6].iloc[0]),
                'bik_recep': str(database[7].iloc[0]),
                'sum': str(database[8].iloc[0]),
                'date': str(database[9].iloc[0]),
                'curr_acc_payer': str(database[10].iloc[0]),
                'corr_acc_payer': str(database[11].iloc[0]),
                'curr_acc_recep': str(database[12].iloc[0]),
                'corr_acc_payer': str(database[13].iloc[0]),
                'proof': proof,
                'previous_hash': previous_hash}
       self.chain.append(block)
       return block
# Эта функция ниже создана для отображения предыдущего блока
   def print_previous_block(self):
       return self.chain[-1]
# Это функция для проверки работы и используется для успешного майнинга блока
   def proof_of_work(self, previous_proof):
       new_proof = 1
       check_proof = False
       while check_proof is False:
           hash_operation = hashlib.sha256(
               str(new_proof**2 - previous_proof**2).encode()).hexdigest()
           if hash_operation[:5] == '00000':
               check_proof = True
           else:
               new_proof += 1
       return new_proof
   def hash(self, block):
       encoded_block = json.dumps(block, sort_keys=True).encode()
       return hashlib.sha256(encoded_block).hexdigest()
   def chain_valid(self, chain):
       previous_block = chain[0]
       block_index = 1
       while block_index < len(chain):
           block = chain[block_index]
           if block['previous_hash'] != self.hash(previous_block):
               return False
           previous_proof = previous_block['proof']
           proof = block['proof']
           hash_operation = hashlib.sha256(
               str(proof**2 - previous_proof**2).encode()).hexdigest()
           if hash_operation[:5] != '00000':
               return False
           previous_block = block
           block_index += 1
       return True

In [ ]:
# Создание веб-приложения с использованием flask
app = Flask(__name__)
# Создаем объект класса blockchain
blockchain = Blockchain()
# Майнинг нового блока
@app.route('/mine_block', methods=['GET'])
def mine_block():
   previous_block = blockchain.print_previous_block()
   previous_proof = previous_block['proof']
   proof = blockchain.proof_of_work(previous_proof)
   previous_hash = blockchain.hash(previous_block)
   block = blockchain.create_block(proof, previous_hash)
   response = {'message': 'A block is MINED',
               'index': block['index'],
               'timestamp': block['timestamp'],
               'proof': block['proof'],
               'previous_hash': block['previous_hash']}
   return jsonify(response), 200
# Отобразить блокчейн в формате json
@app.route('/display_chain', methods=['GET'])
def display_chain():
   response = {'chain': blockchain.chain,
               'length': len(blockchain.chain)}
   return jsonify(response), 200
# Проверка валидности блокчейна
@app.route('/valid', methods=['GET'])
def valid():
   valid = blockchain.chain_valid(blockchain.chain)
   if valid:
       response = {'message': 'The Blockchain is valid.'}
   else:
       response = {'message': 'The Blockchain is not valid.'}
   return jsonify(response), 200
# Запустите сервер flask локально
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Feb/2024 18:17:41] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [12/Feb/2024 18:17:43] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Feb/2024 18:17:47] "GET /main_block HTTP/1.1" 404 -
127.0.0.1 - - [12/Feb/2024 18:17:58] "GET /mine_block HTTP/1.1" 200 -
